In [15]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from PytorchNN import DEVICE, Network

import PytorchNN as net

In [16]:
# Dataset Loading
df = pd.read_csv("data/chess_data_output-110524.csv")
input_features = df.iloc[:, :10].values
output_labels = df.iloc[:, 10:].values
output_labels

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [17]:
# Tensor Conversion
inputs_tensor = torch.tensor(input_features, dtype=torch.float32)
outputs_tensor = torch.tensor(output_labels, dtype=torch.float32)

dataset = TensorDataset(inputs_tensor, outputs_tensor)

batch_size = 20
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [18]:
# Model creation
model = net.Network().to(DEVICE)

In [19]:
%%time
model.train_model(train_loader, epochs=1)

Epoch 1 / 1 - Loss: 1.0925
CPU times: user 1min 6s, sys: 7.24 s, total: 1min 14s
Wall time: 1min 15s


In [20]:
model.test_model(train_loader)

Test Accuracy: 36.45%


In [22]:
with torch.no_grad():
    r = model.predict(torch.tensor([
        [8, 2, 2, 2, 1, 
         8, 2, 2, 2, 1]
    ]).type(torch.float32).to(DEVICE))
r

tensor([[0.3334, 0.3679, 0.2987]], device='mps:0')

In [83]:
import json
# Save the trained weights to JSON
model_weights = {key: value.detach().cpu().numpy().tolist() for key, value in model.state_dict().items()}

with open('exports/model_weights-100epoch-Adam-94k-102424.json', 'w') as f:
    json.dump(model_weights, f)

In [84]:
# Save model with serialization
torch.save(model.state_dict(), "exports/model_state_dict-100epoch-Adam-94k-102424.pt")

In [4]:
# Load model with serialization
model = Network().to(DEVICE)
model.load_state_dict(torch.load("exports/model_state_dict-100epoch-Adam-94k-102424.pt", weights_only=True))
model.eval()

Network(
  (network_layers): Sequential(
    (0): Linear(in_features=10, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=3, bias=True)
    (5): Softmax(dim=1)
  )
  (loss_func): CrossEntropyLoss()
)